In [6]:
from ModelClass import RegressionModel
from DatasetClass import Dataset, DatasetMass
import matplotlib.pyplot as plt
import numpy as np
import itertools
from src.helpers import make_filter_slice
import tensorflow as tf

In [7]:
erik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*H125*.root"
patrik_data = "/scratch/ucjf-atlas/htautau/SM_Htautau_R22/V02_skim_mva_01/*/*/*/*/*Ztt*.root"


In [9]:
dataset = DatasetMass(file_paths=patrik_data, file_name = "data")
dataset.load_data()
print(dataset.train_events)

for feature, target in dataset.train_dataset.take(1):
    print(f"Feature: {feature}")
    print(f"Target: {target}")
    
dataset.augment_data_lorentz(n_slices=1000)

for feature, target in dataset.train_dataset.take(1):
    print(f"Feature: {feature}")
    print(f"Target: {target}")

#train_batches = dataset.train_dataset.batch(32)
#for feature, target in train_batches.take(1):
#    print(f"Feature: {feature.shape}")
#    print(f"Target: {target.shape}")

#print(len(dataset.train_dataset))
#print(len(dataset.dev_dataset))
#print(len(dataset.val_dataset))
#This block of code iterates through the dataset and extracts the pt values of the labels and stores them in a list
#data = [labels.numpy()[0] for features, labels in dataset.train_dataset.take(100000)]

#plt.hist(data, bins=100, range=(50, 130), histtype='step', label='pt distribution', density=False)
#plt.legend(loc='upper right')
#plt.title('pt distribution of the dataset')
#plt.xlabel('pt')
#plt.ylabel('Number of events')
#plt.show()



7799005
Feature: [ 8.6597206e+01  2.1274078e-01 -2.5580318e+00  9.5367432e-07
  8.1345734e+01  8.3852577e-01 -2.2313390e+00  0.0000000e+00
  6.2578499e-01  3.2669282e-01  7.0592850e-01  2.4024684e+02
  1.6794293e+02  2.4598471e+02 -9.1462606e-01  7.6584518e-01
  2.2840584e+01  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  7.4927818e+01  0.0000000e+00 -2.2767708e+00
 -1.8517075e-02  1.0000000e+00  1.0000000e+00  1.0000000e+00
  0.0000000e+00  0.0000000e+00  2.0000000e+00]
Target: 90.89408874511719
Feature: [ 5.1504780e+01  2.5584590e-01 -1.0813713e+00 -2.0363667e+00
  4.0507286e+01  7.1162790e-01  3.6220697e-01 -4.7878933e+00
  4.5152119e-01  1.4435782e+00  1.5125440e+00  9.6761551e+01
  9.2012070e+01  1.3234300e+02 -2.3348835e-01  2.4714675e+00
  1.3950153e+01  8.7358353e+01 -2.9450448e+00 -2.8118095e-01
  1.3864067e+02  6.1242485e+01 -3.4087536e+00  1.8999364e+00
  5.7977106e+02  2.8394463e+01 -3.9452229e-05 -

2025-03-17 00:51:54.535598: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [14]:
dataset.make_slices(n_slices=100)
slices = dataset.slices   

lorentz_mask = tf.constant(dataset.get_lorentz_mask())  # Shape [35] bool values
lorentz_indices = tf.squeeze(tf.where(lorentz_mask), axis=1) # [0 1 2 3 4 5 6 7 13 14 ...] 
n_vectors = tf.shape(lorentz_indices)[0] // 4 # number of 4-vectors
lorentz_indices_4d = tf.reshape(lorentz_indices, (n_vectors, 4))  # [n_vectors, 4]

@tf.function
def augment_lorentz(data, target):
    beta = tf.random.uniform(shape = (), minval=-0.98, maxval=0.98) # Shape ()
    #tf.print("used beta: ", beta)
    gamma = 1.0 / tf.sqrt(1.0 - beta**2) # Shape ()
    #tf.print("used gamma: ", gamma)
    
    for i in range(n_vectors):
        vec_indices = lorentz_indices_4d[i] # Shape (4,)
        pt = data[vec_indices[0]] # scalar values
        eta = data[vec_indices[1]]
        #tf.print("eta: ", eta)
        phi = data[vec_indices[2]]
        E = data[vec_indices[3]]
        #tf.print("E: ", E)
        #print(E.shape)
        
        # Convert to Cartesian coordinates
        #px = pt * tf.cos(phi)
        #py = pt * tf.sin(phi)
        pz = pt * tf.sinh(eta) # Shape ()
        #tf.print("pz: ", pz)

        E_prime = gamma * (E - beta * pz) # Shape ()
        #tf.print("E_prime: ", E_prime)
        pz_prime = gamma * (pz - beta * E) 
        
        epsilon = 1e-8
        eta_prime = tf.asinh(pz_prime / (pt + epsilon)) # Shape ()
        #tf.print("eta_prime: ", eta_prime)
        update_indices = tf.reshape(vec_indices, [-1, 1])  # Shape (4, 1)
        update_values = tf.stack([pt, eta_prime, phi, E_prime]) # Shape (4,)
        # Update the tensor
        data = tf.tensor_scatter_nd_update(
            data,
            indices=update_indices,
            updates=update_values
        )
        #print(data.shape)
        
    return data, target

n_events = 10000
new_dataset = tf.data.Dataset.sample_from_datasets([s.repeat() for s in slices], weights=[1.]*len(slices))
#orig_features, orig_masses = next(iter(new_dataset))
#print(orig_features)
#print(orig_masses)
new_dataset = new_dataset.take(n_events)
augmented_dataset = new_dataset.map(augment_lorentz)
#new_features, new_masses = next(iter(new_dataset))
#print(new_features)š
#print(new_masses)
#batch_dataset = augmented_dataset.batch(n_events)
#print(f"Number of events in batch_dataset: {len(augmented_dataset)}")
#features, masses = next(iter(batch_dataset))

#plt.hist(masses, range=(70, 130), bins=50)
#plt.show()

                                                                    

In [15]:

lorentz_mask = tf.constant(dataset.get_lorentz_mask())  # Shape [35] bool values
print(lorentz_mask)
# Given tensors
features = tf.constant([
    5.56329613e+01, -3.16469967e-01, -9.03481603e-01, 6.74349565e-07,
    5.47039528e+01, -1.49002433e-01, 2.52283901e-01, 1.05658375e-01,
    1.67467535e-01, 1.15576553e+00, 1.16783524e+00, 1.38526611e+02,
    1.10336914e+02, 1.38102692e+02, 1.74350947e-01, 2.87319326e+00,
    1.55200872e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
    0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
    0.00000000e+00, 4.71827660e+01, 0.00000000e+00, -6.81111366e-02,
    -1.58858541e-02, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
    0.00000000e+00, 1.00000000e+00, 1.00000000e+00
], dtype=tf.float32)

label = tf.constant(94.92591, dtype=tf.float32)

# Use from_tensors to keep the pair as a single dataset entry
dataset = tf.data.Dataset.from_tensors((features, label))

# Apply the mapping function
dataset = dataset.map(augment_lorentz)

new_features, new_label = next(iter(dataset))
print(new_features)
print(new_label)

tf.Tensor(
[ True  True  True  True  True  True  True  True False False False False
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False False False False False False], shape=(35,), dtype=bool)
tf.Tensor(
[ 5.56329613e+01 -3.16706300e-01 -9.03481603e-01 -7.03347385e-01
  5.47039528e+01 -1.49041474e-01  2.52283901e-01 -2.15698794e-01
  1.67467535e-01  1.15576553e+00  1.16783524e+00  1.38526611e+02
  1.10336914e+02  1.38102692e+02  1.78831533e-01  2.87319326e+00
  1.64828949e+01  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  4.71827660e+01 -1.32284049e-05 -6.81111366e-02
 -1.58981103e-02  1.00000000e+00  1.00000000e+00  1.00000000e+00
  0.00000000e+00  1.00000000e+00  1.00000000e+00], shape=(35,), dtype=float32)
tf.Tensor(94.92591, shape=(), dtype=float32)


In [15]:
dataset.make_slices(n_slices=10)
slices = dataset.slices 
phi_mask = tf.constant(dataset.get_phi_mask())

@tf.function
def augment_phi(data, target):
    # generate random rotation angle
    angle = tf.random.uniform(shape=(tf.shape(data)[0],), minval=-np.pi, maxval=np.pi)
    #tf.print("angle: ", angle.shape)
    #tf.print("data: ", data.shape)
    # apply rotation
    data  = tf.where(phi_mask, data + angle, data)
    
    # normalize angles between -pi and pi
    data = tf.where(phi_mask, tf.math.atan2(tf.sin(data), tf.cos(data)), data)
    
    return data, target

# sample from the slices
n_events = 1000
new_dataset = tf.data.Dataset.sample_from_datasets([s.repeat() for s in slices], weights=[1.]*len(slices))
new_dataset = new_dataset.take(n_events)

# apply augmentation
new_dataset = new_dataset.map(augment_phi)
#aug_dataset = new_dataset.batch(n_events)


#print(f"Number of events in batch_dataset: {len(batch_dataset)}")
#features, masses = next(iter(batch_dataset))
#print(features.shape)
#print(features)

#plt.hist(masses, range=(70, 130), bins=50)
#plt.show()




In [16]:
from tensorflow.keras.layers import Normalization, Input, Dense
from src.helpers import pick_only_data

normalizer = Normalization()
print("created normalizer")
normalizer.adapt(new_dataset.map(pick_only_data))
print("adapted normalizer")

created normalizer
adapted normalizer


In [ ]:
param_grid = {
    'batch_size': [10],
    'learning_rate': [0.01, 0.1],
    'epochs': [5, 20, 30]
}

iterable = list(itertools.product(*param_grid.values()))
for params in iterable:
    model = RegressionModel(dataset=dataset, batch_size=params[0], initial_learning_rate=params[1], n_epochs=params[2])
    model.prepare_dataset()
    model.create_normalizer()
    model.build_model()
    model.train_model(model.train_batch, model.val_batch)
    model.plot_history(model.history)

Batching datasets...
